## Counter

In [1]:
import magma as m

To implement a counter we can use the `DefineRegister` primitive and the `+` operator. Arithmetic operators are available on the types `UInt` and `SInt`, but not on `Bits`.

In [2]:
import os
os.environ['MANTLE_TARGET'] = 'ice40'
from mantle import DefineRegister

Register4 = DefineRegister(4)
print('{} [{}]'.format(str(Register4), str(Register4.interface)))

class Counter4(m.Circuit):
    IO = ["O", m.Out(m.UInt(4))] + m.ClockInterface()
    @classmethod
    def definition(io):
        reg4 = Register4()
        m.wire(reg4(m.uint(reg4.O) + m.uint(1,4)), io.O)
        m.wireclock(io, reg4)

import mantle lattice ice40
import mantle lattice mantle40
Register4 ["I", In(Bits(4)), "O", Out(Bits(4)), "CLK", In(Clock)]


## Compile and Build

We can compile and inspect the verilog.

In [3]:
m.compile("build/counter", Counter4)

compiling Register4
compiling FullAdder
compiling Add4
compiling Counter4


In [4]:
with open("build/counter.v", "r") as counter_verilog:
    print(counter_verilog.read())

module Register4 (input [3:0] I, output [3:0] O, input  CLK);
wire  inst0_Q;
wire  inst1_Q;
wire  inst2_Q;
wire  inst3_Q;
SB_DFF inst0 (.C(CLK), .D(I[0]), .Q(inst0_Q));
SB_DFF inst1 (.C(CLK), .D(I[1]), .Q(inst1_Q));
SB_DFF inst2 (.C(CLK), .D(I[2]), .Q(inst2_Q));
SB_DFF inst3 (.C(CLK), .D(I[3]), .Q(inst3_Q));
assign O = {inst3_Q,inst2_Q,inst1_Q,inst0_Q};
endmodule

module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I0), .I1(I1), .I2(CIN), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0), .I1(I1), .CI(CIN), .CO(inst1_CO));
assign O = inst0_O;
assign COUT = inst1_CO;
endmodule

module Add4 (input [3:0] I0, input [3:0] I1, output [3:0] O);
wire  inst0_O;
wire  inst0_COUT;
wire  inst1_O;
wire  inst1_COUT;
wire  inst2_O;
wire  inst2_COUT;
wire  inst3_O;
wire  inst3_COUT;
FullAdder inst0 (.I0(I0[0]), .I1(I1[0]), .CIN(1'b0), .O(inst0_O), .COUT(inst0_COUT));
FullAdder inst1 (.I0(I0[1]), .I1(I1[1]